#NASA-Inspired Research Assistant

Copyright 2025, Denis Rothman

**Goal:** This notebook is the final demonstration for Chapter 7, showcasing our fully upgraded Context Engine as a complete, functional application: the **NASA-Inspired Research Assistant**. Here, we will witness the culmination of our work on verifiability and security. The engine will tackle a complex research query, leveraging its new capabilities to produce a precise, citable, and secure response.

This notebook demonstrates the following key capabilities:

* **High-Fidelity RAG in Action:** See the upgraded `Researcher` agent retrieve information from our new knowledge base and programmatically generate citations for its claims.

* **Agent Defenses at Work:** Observe how the `helper_sanitize_input` function provides a silent but essential layer of security, protecting the engine from potentially tainted data.

* **Complex, Dynamic Planning:** Analyze the technical trace to deconstruct the sophisticated, multi-step plan the `Planner` autonomously creates to answer a multi-faceted research question.

* **Verifiable, End-to-End Output:** The final result will be a clear, synthesized answer followed by a list of the source documents used, proving the system is not just powerful, but trustworthy.

# I. Inititalization

## GitHub

In [1]:
# --- Downloading the utilities and the agents ---

# The !curl command is a simple and effective way to download raw files from a public GitHub repo.
# The -L flag ensures that it follows any redirects.

# print("Downloading helper files from public repository...")
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/utils.py --output utils.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/helpers.py --output helpers.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/agents.py --output agents.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/registry.py.py --output registry.py.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/engine.py --output engine.py

# print("✅ Files downloaded successfully!")

In [2]:
import requests
import os
from google.colab import userdata

def download_private_github_file(filename, destination_path="."):
    """
    Downloads a file from a private GitHub repository using a token
    stored in Colab Secrets.

    Requires a secret named 'GITHUB_TOKEN' to be set in the Colab UI.
    """
    # --- Configuration: Replace with your repository details ---
    owner = "Denis2054"
    repo = "Context-Engineering"
    branch = "main"
    # ---------------------------------------------------------

    try:
        # Securely fetch the GitHub token from Colab Secrets
        token = userdata.get('GITHUB_TOKEN')
        if not token:
            raise userdata.SecretNotFoundError("Secret 'GITHUB_TOKEN' not found.")
    except userdata.SecretNotFoundError:
        print("🛑 Error: Secret 'GITHUB_TOKEN' not found.")
        print("Please add your GitHub Personal Access Token to the Colab Secrets Manager.")
        return
    except Exception as e:
        print(f"An error occurred while accessing secrets: {e}")
        return

    # Construct the GitHub API URL for the file
    url = f"https://api.github.com/repos/{owner}/{repo}/contents/{filename}?ref={branch}"

    # Prepare headers for authentication and to request the raw file content
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/vnd.github.v3.raw"
    }

    try:
        # Make the request to the GitHub API
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # This will raise an HTTPError for bad responses (4xx or 5xx)

        # Determine the local filename
        local_filename = os.path.join(destination_path, os.path.basename(filename))

        # Save the file content locally
        with open(local_filename, "wb") as f:
            f.write(response.content)

        print(f"✅ Successfully downloaded '{filename}' to '{local_filename}'")

    except requests.exceptions.HTTPError as e:
        print(f"🛑 Error downloading '{filename}': {e}")
        if e.response.status_code == 404:
            print("   Please check that the owner, repo, file path, and branch are correct.")
        elif e.response.status_code == 401:
            print("   Authentication failed. Please check if your GITHUB_TOKEN is valid and has access to the repo.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


# --- Example Usage: Download your utility files ---
# Ensure you have set 'GITHUB_TOKEN' in your Colab Secrets before running this.

download_private_github_file("commons/ch7/utils.py")
download_private_github_file("commons/ch7/helpers.py")
download_private_github_file("commons/ch7/agents.py")
download_private_github_file("commons/ch7/registry.py")
download_private_github_file("commons/ch7/engine.py")

✅ Successfully downloaded 'commons/ch7/utils.py' to './utils.py'
✅ Successfully downloaded 'commons/ch7/helpers.py' to './helpers.py'
✅ Successfully downloaded 'commons/ch7/agents.py' to './agents.py'
✅ Successfully downloaded 'commons/ch7/registry.py' to './registry.py'
✅ Successfully downloaded 'commons/ch7/engine.py' to './engine.py'


## Installation and client setup

In [3]:
#Installation and Client Setup

# Import the setup functions from your new utility file
import utils

# Run the installation
utils.install_dependencies()

# Initialize the OpenAI and Pinecone clients
client, pc = utils.initialize_clients()

🚀 Installing required packages...
✅ All packages installed successfully.

🔑 Initializing API clients...
   - OpenAI client initialized.
   - Pinecone client initialized.
✅ Clients initialized successfully.


## Context Engine library Import

In [4]:
# 1. Import the hardened helper functions (LLM, Embeddings, Pinecone)
import helpers

# 2. Import the specialist agent functions (Librarian, Researcher, Writer)
import agents

# 3. Import the AGENT_TOOLKIT object that knows about all the agents
from registry import AGENT_TOOLKIT

# 4. Import the main context_engine function that orchestrates the entire process
from engine import context_engine

## Engine Room

In [5]:
# === ENGINE ROOM: The Main Execution Function ===
# This function contains all the logic to run the engine.
# We define it here so our final cell can be very simple.

import logging
import pprint
from IPython.display import display, Markdown

def execute_and_display(goal, config, client, pc):
    """
    Runs the context engine with a given goal and configuration,
    then displays the final output and the technical trace.
    """
    logging.info(f"******** Starting Engine for Goal: '{goal}' **********\n")

    # 1. Run the Context Engine using the provided configuration
    result, trace = context_engine(
        goal,
        client=client,
        pc=pc,
        **config  # Unpack the config dictionary into keyword arguments
    )

    # 2. Display the Final Result for the main reader
    print("--- FINAL OUTPUT ---")
    if result:
        display(Markdown(result))
    else:
        print(f"The engine failed to produce a result. Status: {trace.status}")

    # 3. Display the Technical Trace for the developer/technical reader
    print("\n\n--- TECHNICAL TRACE (for the tech reader) ---")
    if trace:
        print(f"Trace Status: {trace.status}")
        print(f"Total Duration: {trace.duration:.2f} seconds")
        print("Execution Steps:")
        # Use pprint for a clean, readable dictionary output
        pp = pprint.PrettyPrinter(indent=2)
        pp.pprint(trace.steps)

## Control Deck configuration

In [6]:
# 1. Define all configuration variables for this run in a dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

#II.CONTROL DECK

=== CONTROL DECK: Define Goal and Run Engine ===
This is the main interactive cell.
1. Change the 'goal' variable to your desired task.
2. Run this cell.


In [7]:
# === CONTROL DECK: NASA Research Assistant ===

# 1. Define a research goal that requires verifiable, cited answers.
goal = "What are the primary scientific objectives of the Juno mission, and what makes its design unique? Please cite your sources."

# 2. Use the standard configuration
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc)

--- FINAL OUTPUT ---


Juno is NASA’s Jupiter scout doing fast pole-to-pole loops to figure out how the giant formed and evolved—basically clues to how our whole solar system came together (and what that means for giant exoplanets).

What it does:
- Checks how much water Jupiter’s got and whether there’s a core, to test formation theories.
- Reads the atmosphere—composition (including water), temps, cloud motions.
- Maps gravity and magnetic fields to peek at the deep interior.
- Dives into the polar magnetosphere for the full magnetic picture.
- Flies below the worst radiation belts to get clean, global data.

Why it matters:
- Pinning down water and core mass is a big litmus test for how Jupiter formed.
- Gravity/magnetic maps reveal its inner structure and deep processes.
- Polar studies complete the magnetic environment story.
- All together, it sharpens our understanding of early solar system conditions and helps us read giant planets around other stars.



--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 142.56 seconds
Execution Steps:
[ { 'agent': 'Librarian',
    'output': { 'blueprint_json': '{"scene_goal": "Summarize information '
                                  'quickly and casually.", "style_guide": "Use '
                                  'informal language. Keep it brief and '
                                  'engaging. Imagine explaining it to a '
                                  'friend.", "instruction": "Summarize the '
                                  'provided facts using the casual style '
                                  'guide."}'},
    'planned_input': { 'intent_query': 'Concise, citation-rich scientific '
                                       'explainer about a space mission. '
                                       'Structure with brief intro, two '
                                       "sections: 'Primary Scientific "
                                       "Objectives' and 